Sample code to plot candlesticks using plotly, no idea what kde is for

In [31]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy import stats, signal 
import plotly.express as px 
import plotly.graph_objects as go

## Heikin-Ashi 

##### https://www.investopedia.com/terms/h/heikinashi.asp

In [32]:
# Function to plot Heikin-Ashi candlestick chart
def plot_heikin_ashi(data):
    fig = go.Figure(data=[go.Candlestick(
    x=data.index, 
    open=heikin_ashi['Open'], 
    high=heikin_ashi['High'], 
    low=heikin_ashi['Low'], 
    close=heikin_ashi['Close'], 
    increasing_line_color='green', 
    decreasing_line_color='red', 
    name='Heikin-Ashi'
    )])
    return fig

In [62]:
# https://plotly.com/python/creating-and-updating-figures/

# Function to create distribution plot
def get_dist_plot(c, v, kx, ky):
    fig = go.Figure()
    fig.add_trace(go.Histogram(name='Vol Profile', 
                               x=c, 
                               y=v, 
                               nbinsx=150,
                               histfunc='sum', 
                               histnorm='probability density',
                               marker_color='#B0C4DE')) 
    fig.add_trace(go.Scatter(name='KDE', x=kx, y=ky, mode='lines', marker_color='#D2691E'))
    return fig

### Parameters to plot

In [34]:
# Fetch OHLCV data using yfinance
ticker = 'AAPL' 
start_date = '2013-01-01' 
end_date = '2024-08-12' 
interval = '1d'

In [35]:
data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

[*********************100%%**********************]  1 of 1 completed


In [36]:
# Extract volume and close prices
volume = data['Volume'] 
close = data['Close']

In [37]:
# Heikin-Ashi calculation
heikin_ashi = pd.DataFrame(index=data.index, columns=['Open', 'High', 'Low', 'Close']) 
heikin_ashi['Close'] = (data['Open'] + data['High'] + data['Low'] + data['Close']) / 4 
heikin_ashi['Open'] = (data['Open'].shift(1) + data['Close'].shift(1)) / 2 
heikin_ashi['High'] = data[['High', 'Open', 'Close']].max(axis=1)
heikin_ashi['Low'] = data[['Low', 'Open', 'Close']].min(axis=1)
heikin_ashi.iloc[0, heikin_ashi.columns.get_loc('Open')] = data.iloc[0]['Open'] # Set the first Open value


In [38]:
heikin_ashi

,Open,High,Low,Close
Date,,,,
2013-01-02,19.779285,19.821428,19.343929,19.638214
2013-01-03,19.693749,19.631071,19.321428,19.470089
2013-01-04,19.463928,19.236786,18.779642,19.003839
2013-01-07,18.999464,18.903570,18.400000,18.664285
2013-01-08,18.676785,18.996071,18.616072,18.818392
...,...,...,...,...
2024-08-05,219.504997,213.500000,196.000000,204.465000
2024-08-06,204.180000,209.990005,201.070007,205.897503
2024-08-07,206.264999,213.639999,206.389999,209.187500


In [39]:
# Create a histogram of volume vs. close
px.histogram(data, x=volume, y=close, nbins=150, orientation='h').show()


In [40]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html

#https://numpy.org/doc/stable/reference/generated/numpy.linspace.html

# KDE calculation
kde_factor = 0.05
num_samples = 5000
kde = stats.gaussian_kde(close, weights=volume, bw_method=kde_factor) 
xr = np.linspace(close.min(), close.max(), num_samples)
kdy = kde(xr)
ticks_per_sample = (xr.max() - xr.min()) / num_samples

In [41]:
kde

In [42]:
xr

array([ 13.94750023,  13.99168357,  14.03586691, ..., 234.73164065,
       234.77582399, 234.82000732])

In [43]:
kdy

array([2.12188025e-02, 2.15393993e-02, 2.18599511e-02, ...,
       6.14811904e-05, 6.09524340e-05, 6.04177751e-05])

In [44]:
# Show distribution plot with KDE
get_dist_plot(close, volume, xr, kdy).show()

In [45]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html

# Find peaks in KDE
peaks, _ = signal.find_peaks(kdy) 
pkx = xr[peaks]
pky = kdy[peaks]

In [46]:
pkx

array([ 17.3938006 ,  26.89321829,  42.13646993,  69.08830616,
        78.19007381,  91.62180859, 116.27611124, 125.28951221,
       131.47517954, 148.04393132, 173.18425069, 190.45993589,
       209.37040459])

In [47]:
pky

array([0.0370311 , 0.02752017, 0.01022061, 0.00237018, 0.00220375,
       0.00084401, 0.00206469, 0.00262208, 0.00239611, 0.00308495,
       0.00271974, 0.00113388, 0.00034508])

In [48]:
# Marker arguments for peaks
pk_marker_args = dict(size=10)

In [49]:
# Plot peaks on distribution plot
fig = get_dist_plot(close, volume, xr, kdy)
fig.add_trace(go.Scatter(name="Peaks", x=pkx, y=pky, mode='markers', marker=pk_marker_args)) 
fig.show()

In [50]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html

# Find peaks with prominence
min_prom = 0
peaks, peak_props = signal.find_peaks(kdy, prominence=min_prom) 
pkx = xr[peaks]
pky = kdy[peaks]

In [51]:
kdy.shape

(5000,)

In [52]:
pkx

array([ 17.3938006 ,  26.89321829,  42.13646993,  69.08830616,
        78.19007381,  91.62180859, 116.27611124, 125.28951221,
       131.47517954, 148.04393132, 173.18425069, 190.45993589,
       209.37040459])

In [53]:
pky

array([0.0370311 , 0.02752017, 0.01022061, 0.00237018, 0.00220375,
       0.00084401, 0.00206469, 0.00262208, 0.00239611, 0.00308495,
       0.00271974, 0.00113388, 0.00034508])

In [54]:
# Plot with prominence lines
fig = get_dist_plot(close, volume, xr, kdy)
fig.add_trace(go.Scatter(name='Peaks', x=pkx, y=pky, mode='markers', marker=pk_marker_args))

# Draw prominence lines
left_base = peak_props['left_bases'] 
right_base = peak_props['right_bases'] 
line_x = pkx
line_y0 = pky
line_y1 = pky - peak_props['prominences']

for x, y0, y1 in zip(line_x, line_y0, line_y1): 
    fig.add_shape(type='line',xref='x', yref='y',x0=x, y0=y0, x1=x, y1=y1, line=dict(color='red', width=2))
fig.show()

In [55]:
# Analyze peak widths
width_range = 1
peaks, peak_props = signal.find_peaks(kdy, prominence=min_prom, width=width_range)

left_ips = peak_props['left_ips']
right_ips = peak_props['right_ips']
width_x0 = xr.min() + (left_ips * ticks_per_sample) 
width_x1 = xr.min() + (right_ips * ticks_per_sample) 
width_y = peak_props['width_heights']

fig = get_dist_plot(close, volume, xr, kdy)
fig.add_trace(go.Scatter(name='Peaks', x=pkx, y=pky, mode='markers', marker=pk_marker_args))

for x0, x1, y in zip(width_x0, width_x1, width_y): 
    fig.add_shape(type='line',
                  xref='x', 
                  yref='y',
                  x0=x0, 
                  y0=y, 
                  x1=x1, 
                  y1=y, 
                  line=dict(color='red', width=2)
                  )

fig.show()

In [56]:
# Adjust peak detection settings for trading
pipsize = 0.0001
max_width_pips = 20
min_prom = kdy.max() * 0.3
width_range = (1, max_width_pips * pipsize / ticks_per_sample)
peaks, peak_props = signal.find_peaks(kdy, width=width_range, prominence=min_prom) 
pkx = xr[peaks]
pky = kdy[peaks]

In [57]:
pkx

array([], dtype=float64)

In [58]:
pky

array([], dtype=float64)

In [59]:
# Integrate area under KDE curve for each peak
left_base = peak_props['left_bases']
right_base = peak_props['right_bases']
int_from = xr.min() + (left_base * ticks_per_sample) 
int_to = xr.min() + (right_base * ticks_per_sample)

integrated_values = [kde.integrate_box_1d(x0, x1) for x0, x1 in zip(int_from, int_to)] 
print(integrated_values)

[]


In [60]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,19.779285,19.821428,19.343929,19.608213,16.705696,560518000
2013-01-03,19.567142,19.631071,19.321428,19.360714,16.494843,352965200
2013-01-04,19.177500,19.236786,18.779642,18.821428,16.035379,594333600
2013-01-07,18.642857,18.903570,18.400000,18.710714,15.941051,484156400
2013-01-08,18.900356,18.996071,18.616072,18.761070,15.983956,458707200
...,...,...,...,...,...,...
2024-08-05,199.089996,213.500000,196.000000,209.270004,209.028061,119548600
2024-08-06,205.300003,209.990005,201.070007,207.229996,206.990402,69660500
2024-08-07,206.899994,213.639999,206.389999,209.820007,209.577423,63516400


In [64]:
# Combine Heikin-Ashi candlestick chart with existing plots
fig_ha = plot_heikin_ashi(data) 
fig_ha.show()

In [65]:
fig.data

(Histogram({
     'histfunc': 'sum',
     'histnorm': 'probability density',
     'marker': {'color': '#B0C4DE'},
     'name': 'Vol Profile',
     'nbinsx': 150,
     'x': array([ 19.60821342,  19.36071396,  18.8214283 , ..., 209.82000732,
                 213.30999756, 216.24000549]),
     'y': array([560518000, 352965200, 594333600, ...,  63516400,  47161100,  42201600])
 }),
 Scatter({
     'marker': {'color': '#D2691E'},
     'mode': 'lines',
     'name': 'KDE',
     'x': array([ 13.94750023,  13.99168357,  14.03586691, ..., 234.73164065,
                 234.77582399, 234.82000732]),
     'y': array([2.12188025e-02, 2.15393993e-02, 2.18599511e-02, ..., 6.14811904e-05,
                 6.09524340e-05, 6.04177751e-05])
 }),
 Scatter({
     'marker': {'size': 10},
     'mode': 'markers',
     'name': 'Peaks',
     'x': array([ 17.3938006 ,  26.89321829,  42.13646993,  69.08830616,  78.19007381,
                  91.62180859, 116.27611124, 125.28951221, 131.47517954, 148.04393132,
   

In [61]:
# Combine Heikin-Ashi candlestick chart with existing plots
fig_ha = plot_heikin_ashi(data) 
for trace in fig.data:
    fig_ha.add_trace(trace) 
    
fig_ha.show()